# Computaional Theory Problems

## Problem 1: Binary Words and Operations   

### Write helper to secure 32-bit

In [32]:
import numpy as np
from typing import Union

Word = np.uint32

def _to_u32(x: Union[int, np.integer]) -> Word:
    """Force value to 32-bit unsigned word."""
    return Word(int(x) & 0xFFFFFFFF)

def _rotr(x: Word, n: int) -> Word:
    """Rotate-right 32-bit x by n."""
    x = _to_u32(x); n = int(n) % 32
    if n == 0:
        return x
    return _to_u32((x >> n) | (x << Word(32 - n)))

def _shr(x: Word, n: int) -> Word:
    """Logical right shift 32-bit x by n (zero-fill)."""
    x = _to_u32(x); n = int(n) % 32
    return _to_u32(x >> n)

### Implement Parity / Ch / Maj

In [33]:
def Parity(x, y, z) -> Word:
    """Parity(x,y,z): XOR from 32-bit ( SHA-1 parity)."""
    return _to_u32(_to_u32(x) ^ _to_u32(y) ^ _to_u32(z))

def Ch(x, y, z) -> Word:
    """Ch (Eq. 4.2): (x & y) ^ (~x & z) — pick bit from y if bit x =1, otherwise z."""
    x, y, z = _to_u32(x), _to_u32(y), _to_u32(z)
    return _to_u32((x & y) ^ ((~x) & z))

def Maj(x, y, z) -> Word:
    """Maj (Eq. 4.3): (x & y) ^ (x & z) ^ (y & z) — majority base on each bit."""
    x, y, z = _to_u32(x), _to_u32(y), _to_u32(z)
    return _to_u32((x & y) ^ (x & z) ^ (y & z))


### Implement Big Sigma (Σ0, Σ1)

In [34]:
def Sigma0(x) -> Word:
    r"""Σ0_256 (Eq. 4.4): ROTR^2(x) ^ ROTR^13(x) ^ ROTR^22(x)"""
    x = _to_u32(x)
    return _to_u32(_rotr(x, 2) ^ _rotr(x, 13) ^ _rotr(x, 22))

def Sigma1(x) -> Word:
    r"""Σ1_256 (Eq. 4.5): ROTR^6(x) ^ ROTR^11(x) ^ ROTR^25(x)"""
    x = _to_u32(x)
    return _to_u32(_rotr(x, 6) ^ _rotr(x, 11) ^ _rotr(x, 25))


### Implement Small sigma (σ0, σ1)

In [35]:
def sigma0(x) -> Word:
    r"""σ0_256 (Eq. 4.6): ROTR^7(x) ^ ROTR^18(x) ^ SHR^3(x)"""
    x = _to_u32(x)
    return _to_u32(_rotr(x, 7) ^ _rotr(x, 18) ^ _shr(x, 3))

def sigma1(x) -> Word:
    r"""σ1_256 (Eq. 4.7): ROTR^17(x) ^ ROTR^19(x) ^ SHR^10(x)"""
    x = _to_u32(x)
    return _to_u32(_rotr(x, 17) ^ _rotr(x, 19) ^ _shr(x, 10))


### Writing test case

In [36]:
# --- Predefined 32-bit input values for demonstration ---
x = np.uint32(0x6a09e667)  # example constant from SHA-256 initial hash values
y = np.uint32(0x12345678)  # arbitrary test value
z = np.uint32(0xdeadbeef)  # another arbitrary test value

# --- Display input values ---
print("===== INPUT VALUES =====")
print(f"x = {hex(int(x))}")
print(f"y = {hex(int(y))}")
print(f"z = {hex(int(z))}")

# --- Logical operation outputs ---
# These functions define the logical core of SHA-256 as per FIPS PUB 180-4 section 4.1.2
print("\n===== LOGIC FUNCTIONS =====")
print(f"Parity(x, y, z) = {hex(int(Parity(x, y, z)))}   # Bitwise XOR of all three")
print(f"Ch(x, y, z)     = {hex(int(Ch(x, y, z)))}   # Choose: (x & y) ^ (~x & z)")
print(f"Maj(x, y, z)    = {hex(int(Maj(x, y, z)))}   # Majority: (x & y) ^ (x & z) ^ (y & z)")

# --- Sigma functions ---
# These are the rotation and shift based functions used in SHA-256 compression
print("\n===== SIGMA FUNCTIONS =====")
print(f"Sigma0(x) = {hex(int(Sigma0(x)))}   # Σ0: ROTR2 ^ ROTR13 ^ ROTR22")
print(f"Sigma1(x) = {hex(int(Sigma1(x)))}   # Σ1: ROTR6 ^ ROTR11 ^ ROTR25")
print(f"sigma0(x) = {hex(int(sigma0(x)))}   # σ0: ROTR7 ^ ROTR18 ^ SHR3")
print(f"sigma1(x) = {hex(int(sigma1(x)))}   # σ1: ROTR17 ^ ROTR19 ^ SHR10")

===== INPUT VALUES =====
x = 0x6a09e667
y = 0x12345678
z = 0xdeadbeef

===== LOGIC FUNCTIONS =====
Parity(x, y, z) = 0xa6900ef0   # Bitwise XOR of all three
Ch(x, y, z)     = 0x96a45ee8   # Choose: (x & y) ^ (~x & z)
Maj(x, y, z)    = 0x5a2df66f   # Majority: (x & y) ^ (x & z) ^ (y & z)

===== SIGMA FUNCTIONS =====
Sigma0(x) = 0xce20b47e   # Σ0: ROTR2 ^ ROTR13 ^ ROTR22
Sigma1(x) = 0x55b65510   # Σ1: ROTR6 ^ ROTR11 ^ ROTR25
sigma0(x) = 0xba0cf582   # σ0: ROTR7 ^ ROTR18 ^ SHR3
sigma1(x) = 0xcfe5da3c   # σ1: ROTR17 ^ ROTR19 ^ SHR10


## Problem 2: Fractional Parts of Cube Roots

In [37]:
1

1

## Problem 3: Padding

In [38]:
1

1

## Problem 4: Hashes

In [39]:
1

1

## Problem 5: Passwords

In [40]:
1

1